Imports

In [ ]:
import sys
import os
from google.colab import drive
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as Backend
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras import models

Access Drive

In [ ]:

drive.mount('/content/drive', force_remount=True)
!ls "/content/drive/My Drive/Colab Notebooks/apple_classifier"

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

class Apple_Neural_Network_Classifier:


  # sys.path.insert(0, TRAINING_DATA_DIR)
  # sys.path.insert(0, TEST_DATA_DIR)ZA
  # sys.path.insert(1, OUTPUT_DIR)
  # Paths. Dont forget about the tail slash "/"
  TRAINING_DATA_DIR = "/content/drive/My Drive/Colab Notebooks/apple_classifier/training/"
  VALIDATION_DATA_DIR = "/content/drive/My Drive/Colab Notebooks/apple_classifier/validation/"
  TEST_DATA_DIR = "/content/drive/My Drive/Colab Notebooks/apple_classifier/Test/"
  OUTPUT_DIR = "/content/drive/My Drive/Colab Notebooks/apple_classifier/output/"
  training_data = TRAINING_DATA_DIR
  validation_data = VALIDATION_DATA_DIR
  # Need to be ordered same as it is in the folder
  apples_list = ['blotch', 'normal', 'rot', 'scab']

  # Params
  epochs = 10
  image_size = (100, 100)
  batch_size = 10
  steps = 32
  validation_steps = 200
  filters_conv1 = 32
  filters_conv2 = 64
  filter_size1 = (3, 3)
  filter_size2 = (2, 2)
  max_pool = (2, 2)
  classes = 4  # Number of categories to predict/classify. This case 4.
  learning_rate = 0.0005
  image_channels = 3

  def __init__(self):
    pass


  def image_preprocessing(self):
    # Image Pre-processing
    training_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.3,
        zoom_range = 0.3,
        horizontal_flip = True
        )

    validation_datagen = ImageDataGenerator(
        rescale = 1./255
        )
    training_image = training_datagen.flow_from_directory(
        self.training_data,
        target_size = self.image_size,
        batch_size = self.batch_size,
        class_mode = 'categorical'
        )

    validation_image = validation_datagen.flow_from_directory(
        self.validation_data,
        target_size = self.image_size,
        batch_size = self.batch_size,
        class_mode = 'categorical'
        )
    
    return training_image, validation_image
    

  def train(self):

    # Preprocess images
    training_image, validation_image = self.image_preprocessing()

    # CNN Creation using Sequential

    neural_net = Sequential()

    # 1st layer
    neural_net.add(Convolution2D(
        self.filters_conv1, 
        self.filter_size1, 
        padding = 'same',
        input_shape = (self.image_size[0], self.image_size[1], self.image_channels),
        activation = 'relu'
        ))
    neural_net.add(MaxPooling2D(pool_size = self.max_pool))

    # 2nd layer
    neural_net.add(Convolution2D(
        self.filters_conv2, 
        self.filter_size2, 
        padding = 'same',
        activation = 'relu'
        ))
    neural_net.add(MaxPooling2D(pool_size = self.max_pool))

    neural_net.add(Flatten())
    neural_net.add(Dense(256, activation = 'relu'))
    neural_net.add(Dropout(0.5))
    neural_net.add(Dense(self.classes, activation = 'softmax'))

    neural_net.compile(
        loss = 'categorical_crossentropy', 
        # optimizer = optimizers.Adam(lr = self.learning_rate), 
        optimizer = 'adam',
        run_eagerly = True,
        metrics = ['accuracy']
        )
    
    neural_net.fit(
        training_image, 
        steps_per_epoch = self.steps, 
        epochs = self.epochs, 
        validation_data = validation_image, 
        validation_steps = self.validation_steps
        )
    
    if not os.path.exists(self.OUTPUT_DIR):
      os.mkdir(self.OUTPUT_DIR)
    
    neural_net.save('{}/apple_model.h5'.format(self.OUTPUT_DIR))
    neural_net.save_weights('{}/apple_weights.h5'.format(self.OUTPUT_DIR))

  def predict(self, file_to_predict):
    file_model = '{}/apple_model.h5'.format(self.OUTPUT_DIR)
    file_weights = '{}/apple_weights.h5'.format(self.OUTPUT_DIR)

    neural_net = tf.keras.models.load_model(file_model)
    neural_net.load_weights(file_weights)
    
    image = img_to_array(
        load_img("{}/{}".format(self.TEST_DATA_DIR, file_to_predict), target_size=self.image_size)
    )
    image = np.expand_dims(image, axis=0)
    prediction = neural_net.predict(image) # eg. [[1, 0, 0, 0]]

    return np.argmax(prediction[0])

  def get_apple_label(self, index):
    try:
      label = self.apples_list[index]
    except IndexError:
      label = "No label detected"

    return label

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
output	Test  training	validation


Train CNN

In [ ]:

# Clear session Keras
Backend.clear_session()

# Train
classifier = Apple_Neural_Network_Classifier()
classifier.train()

Found 319 images belonging to 4 classes.
Found 319 images belonging to 4 classes.
Epoch 1/10
32/32 [==============================] - 11s 346ms/step - loss: 1.4840 - accuracy: 0.3950 - val_loss: 0.9841 - val_accuracy: 0.5361
Epoch 2/10
32/32 [==============================] - 11s 345ms/step - loss: 1.0562 - accuracy: 0.5016 - val_loss: 0.8975 - val_accuracy: 0.6144
Epoch 3/10
32/32 [==============================] - 11s 344ms/step - loss: 0.9242 - accuracy: 0.5799 - val_loss: 0.7589 - val_accuracy: 0.7085
Epoch 4/10
32/32 [==============================] - 11s 344ms/step - loss: 0.8946 - accuracy: 0.6176 - val_loss: 0.6904 - val_accuracy: 0.7273
Epoch 5/10
32/32 [==============================] - 11s 353ms/step - loss: 0.8296 - accuracy: 0.6458 - val_loss: 0.7398 - val_accuracy: 0.7179
Epoch 6/10
32/32 [==============================] - 11s 345ms/step - loss: 0.7938 - accuracy: 0.6771 - val_loss: 0.5854 - val_accuracy: 0.7900
Epoch 7/10
32/32 [==============================] - 11s 345m

Prediction with CNN

In [ ]:
# Clear session Keras
Backend.clear_session()

# Classify
classifier = Apple_Neural_Network_Classifier()

img_dir = classifier.TEST_DATA_DIR
count = 0
passed = 0
failed = 0
for f1 in os.listdir(img_dir):
    # file_name = os.path.join(img_dir, f1)
    res = classifier.get_apple_label(classifier.predict(f1))
    class_type = f1.split()
    count += 1
    if res == class_type[0]:
      passed_failed = "pass"
      passed += 1
    else:
      passed_failed = "failed"
      failed += 1
    print("{}.- {} -> {} = {}".format(count, f1, res, passed_failed))

print("Summary: {} passed and {} failed of total {}.".format(passed, failed, count))
print("Accuracy: {}%".format(passed * 100 // count))
print("Done")

1.- blotch (66).jpg -> blotch = pass
2.- blotch (68).jpg -> blotch = pass
3.- blotch (67).jpg -> blotch = pass
4.- blotch (69).jpg -> blotch = pass
5.- blotch (70).jpg -> blotch = pass
6.- blotch (71).jpg -> scab = failed
7.- blotch (74).jpg -> blotch = pass
8.- blotch (72).jpg -> blotch = pass
9.- blotch (73).jpg -> blotch = pass
10.- blotch (75).jpg -> blotch = pass
11.- blotch (76).jpg -> blotch = pass
12.- blotch (77).jpg -> blotch = pass
13.- blotch (78).jpg -> rot = failed
14.- blotch (79).jpg -> rot = failed
15.- blotch (80).jpg -> blotch = pass
16.- rot (66).jpg -> blotch = failed
17.- rot (67).jpg -> scab = failed
18.- rot (68).jpg -> scab = failed
19.- rot (69).jpg -> rot = pass
20.- rot (70).jpg -> rot = pass
21.- rot (71).jpg -> rot = pass
22.- rot (72).jpg -> rot = pass
23.- rot (73).jpg -> scab = failed
24.- rot (74).jpg -> blotch = failed
25.- rot (75).jpg -> blotch = failed
26.- rot (76).jpg -> rot = pass
27.- rot (77).jpg -> rot = pass
28.- rot (78).jpg -> blotch = fai